In [1]:
import torch
from torchinfo import summary
from model import model_builder

/home/gkmo/workspace/rgbd-yolov2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model = ln.models.Darknet19(1000)
# model.load('weights/darknet19_448.weights')

# # Save as PyTorch weight file (Not strictly necessary, but it is faster than darknet weight files)
# model.save('weights/darknet19_448.pt')

# # Converting Darknet19 weights to Yolo (This is the same as the darknet19_448.conv.23.weights from darknet)
# model.save('weights/yolo-pretrained_darknet.pt', remap=ln.models.YoloV2.remap_darknet19)

# # Load yolo weights (Requires `strict=False`, because not all layers have weights in this file)
# detection_model = ln.models.YoloV2(3)
# detection_model.load('weights/yolo-pretrained_darknet.pt', strict=False)

## Test the weights of rgbd network

In [2]:
rgb_model = model_builder(num_classes=3, model_type="rgb")
# summary(model=rgb_model, 
#         input_size=(32, 3, 416, 416), # make sure this is "input_size", not "input_shape"
#         # col_names=["input_size"], # uncomment for smaller output
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"])

/home/gkmo/workspace/rgbd-yolov2/.venv/lib/python3.10/site-packages/lightnet/network/module/_lightnet.py:293: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.loa

In [3]:
depth_pretrained = torch.load("models/depth_99.pth")

/tmp/ipykernel_6681/3341212066.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  depth_pretrained = torch.load("models/depth_99.pth")


In [4]:
fusion_model = model_builder(num_classes=3, model_type="rgbd", fuse_layer=15)

Weights equivalent


In [5]:
input_tensor = torch.randn(32, 3, 416, 416)
output_tensor = rgb_model(input_tensor)
print(output_tensor.shape)

torch.Size([32, 40, 13, 13])


In [17]:
rgb_model.state_dict()["backbone.module.1_convbatch.layers.1.num_batches_tracked"]

tensor(1)

In [24]:
#fusion_model.state_dict().keys()

for key, value in rgb_model.state_dict().items():
    print(key)

backbone.module.1_convbatch.layers.0.weight
backbone.module.1_convbatch.layers.1.weight
backbone.module.1_convbatch.layers.1.bias
backbone.module.1_convbatch.layers.1.running_mean
backbone.module.1_convbatch.layers.1.running_var
backbone.module.1_convbatch.layers.1.num_batches_tracked
backbone.module.3_convbatch.layers.0.weight
backbone.module.3_convbatch.layers.1.weight
backbone.module.3_convbatch.layers.1.bias
backbone.module.3_convbatch.layers.1.running_mean
backbone.module.3_convbatch.layers.1.running_var
backbone.module.3_convbatch.layers.1.num_batches_tracked
backbone.module.5_convbatch.layers.0.weight
backbone.module.5_convbatch.layers.1.weight
backbone.module.5_convbatch.layers.1.bias
backbone.module.5_convbatch.layers.1.running_mean
backbone.module.5_convbatch.layers.1.running_var
backbone.module.5_convbatch.layers.1.num_batches_tracked
backbone.module.6_convbatch.layers.0.weight
backbone.module.6_convbatch.layers.1.weight
backbone.module.6_convbatch.layers.1.bias
backbone.mod

In [22]:
rgbd_regular_weight = fusion_model.state_dict()['layers.1.regular.11_convbatch.layers.1.weight'].cpu()
rgb_weight = rgb_model.state_dict()['backbone.module.11_convbatch.layers.1.weight'].cpu()

In [30]:
depth_weight = depth_pretrained['backbone.module.10_convbatch.layers.1.weight'].cpu()
rgbd_fusion_fusion_weight = fusion_model.state_dict()['layers.1.fusion.10_convbatch.layers.1.weight'].cpu()

In [31]:
torch.equal(depth_weight, rgbd_fusion_fusion_weight)

True

In [32]:
torch.equal(rgbd_regular_weight, rgb_weight)

True